<a href="https://colab.research.google.com/github/njokinjuguna/Machine-learning-Models/blob/main/Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# !pip install transformers
# !pip install transformers torch datasets
# !pip install git+https://github.com/openai/CLIP.git

In [36]:
import os
from PIL import Image
import torch
import clip
from transformers import CLIPProcessor, CLIPModel
from sklearn.metrics.pairwise import cosine_similarity

In [37]:
# Load CLIP model
model_name = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)

In [39]:
# Folder path for kitchen designs (change this to your folder path)
kitchen_folder = "kitchen/"  # Replace with your kitchen folder path
kitchen_images = [os.path.join(kitchen_folder, fname) for fname in os.listdir(kitchen_folder) if fname.endswith('.jpg')]

In [40]:
# Client's description
client_input = "modern kitchen with a large island"
texts = [client_input]

In [41]:
# Calculate similarity scores for each image in the kitchen folder
similarities = []
for image_path in kitchen_images:
    image = Image.open(image_path)
    inputs = processor(text=texts, images=image, return_tensors="pt", padding=True)

    with torch.no_grad():
        outputs = model(**inputs)
        image_features = outputs.image_embeds
        text_features = outputs.text_embeds

    # Compute cosine similarity
    similarity = cosine_similarity(image_features.cpu(), text_features.cpu())[0][0]
    similarities.append((image_path, similarity))

# Sort the images by similarity score (highest score first)
similarities.sort(key=lambda x: x[1], reverse=True)

# Print the best matching image and score
best_match_image = similarities[0][0]
print(f"Best matching design: {best_match_image} with score: {similarities[0][1]}")


IndexError: list index out of range